In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/My Drive/TrueCue/'

Mounted at /content/drive
/content/drive/My Drive/TrueCue


In [2]:
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle

## Import latest Covid data from Our World in Data

In [3]:
df=pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv') 
df.tail().T

,51161,51162,51163,51164,51165
iso_code,NaN,NaN,NaN,NaN,NaN
continent,NaN,NaN,NaN,NaN,NaN
location,International,International,International,International,International
date,2020-10-15,2020-10-16,2020-10-17,2020-10-18,2020-10-19
total_cases,696,696,696,696,696
new_cases,NaN,NaN,NaN,NaN,NaN
new_cases_smoothed,NaN,NaN,NaN,NaN,NaN
total_deaths,7,7,7,7,7
new_deaths,NaN,NaN,NaN,NaN,NaN
new_deaths_smoothed,NaN,NaN,NaN,NaN,NaN


## Extract new_cases: new confirmed cases of COVID-19

In [4]:
df_pvt=df.pivot(index='date', columns='iso_code', values='new_cases').fillna(0.)
df_pvt.tail()

iso_code,NaN,ABW,AFG,AGO,AIA,ALB,AND,ARE,ARG,ARM,ATG,AUS,AUT,AZE,BDI,BEL,BEN,BES,BFA,BGD,BGR,BHR,BHS,BIH,BLR,BLZ,BMU,BOL,BRA,BRB,BRN,BTN,BWA,CAF,CAN,CHE,CHL,CHN,CIV,CMR,...,SLV,SMR,SOM,SRB,SSD,STP,SUR,SVK,SVN,SWE,SWZ,SXM,SYC,SYR,TCA,TCD,TGO,THA,TJK,TLS,TTO,TUN,TUR,TWN,TZA,UGA,UKR,URY,USA,UZB,VAT,VCT,VEN,VGB,VIR,VNM,YEM,ZAF,ZMB,ZWE
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-10-15,0.0,26.0,32.0,192.0,0.0,203.0,195.0,1431.0,14932.0,0.0,1.0,24.0,1606.0,369.0,0.0,8216.0,67.0,0.0,0.0,1684.0,819.0,349.0,222.0,482.0,597.0,34.0,0.0,219.0,27235.0,5.0,1.0,3.0,0.0,0.0,2506.0,2724.0,1092.0,11.0,34.0,108.0,...,286.0,0.0,0.0,245.0,0.0,0.0,11.0,1410.0,707.0,970.0,19.0,10.0,0.0,57.0,0.0,8.0,11.0,13.0,39.0,0.0,27.0,2234.0,1671.0,0.0,0.0,124.0,5590.0,1.0,59386.0,316.0,0.0,0.0,614.0,0.0,0.0,9.0,1.0,1877.0,29.0,19.0
2020-10-16,0.0,30.0,0.0,166.0,0.0,257.0,0.0,1398.0,17096.0,1371.0,0.0,21.0,1396.0,530.0,0.0,6266.0,18.0,0.0,30.0,1600.0,914.0,333.0,0.0,569.0,613.0,63.0,0.0,178.0,28523.0,3.0,0.0,0.0,203.0,0.0,2345.0,2690.0,1124.0,36.0,40.0,0.0,...,295.0,7.0,0.0,203.0,9.0,0.0,11.0,1929.0,746.0,596.0,18.0,8.0,1.0,48.0,0.0,21.0,13.0,4.0,38.0,0.0,40.0,0.0,1693.0,5.0,0.0,48.0,5062.0,79.0,63785.0,309.0,7.0,0.0,464.0,0.0,1.0,2.0,0.0,1770.0,43.0,20.0
2020-10-17,0.0,4.0,47.0,250.0,0.0,289.0,187.0,1412.0,16546.0,1465.0,0.0,9.0,1691.0,509.0,2.0,979.0,0.0,0.0,8.0,1527.0,998.0,371.0,132.0,621.0,658.0,46.0,0.0,243.0,30914.0,1.0,0.0,0.0,0.0,0.0,2374.0,3087.0,1694.0,20.0,18.0,0.0,...,204.0,0.0,0.0,265.0,10.0,3.0,19.0,2075.0,836.0,0.0,13.0,9.0,0.0,56.0,1.0,11.0,31.0,10.0,40.0,0.0,47.0,0.0,1812.0,0.0,0.0,217.0,5992.0,33.0,70256.0,100.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2019.0,0.0,24.0
2020-10-18,0.0,15.0,68.0,126.0,0.0,273.0,0.0,1538.0,13510.0,1540.0,7.0,12.0,2040.0,528.0,5.0,0.0,0.0,0.0,0.0,1209.0,603.0,246.0,111.0,0.0,0.0,47.0,0.0,148.0,24062.0,0.0,0.0,9.0,0.0,0.0,2215.0,0.0,1813.0,30.0,26.0,0.0,...,191.0,0.0,0.0,227.0,0.0,0.0,10.0,1968.0,1624.0,0.0,19.0,3.0,0.0,46.0,1.0,4.0,22.0,7.0,41.0,0.0,40.0,5752.0,1723.0,0.0,0.0,121.0,6410.0,51.0,56611.0,540.0,0.0,2.0,289.0,0.0,6.0,2.0,2.0,1928.0,130.0,11.0
2020-10-19,0.0,18.0,59.0,240.0,0.0,281.0,0.0,1215.0,10561.0,1694.0,0.0,7.0,1062.0,647.0,6.0,0.0,0.0,0.0,38.0,1274.0,395.0,331.0,75.0,716.0,1306.0,38.0,0.0,61.0,10982.0,3.0,0.0,2.0,0.0,0.0,1827.0,195.0,1757.0,17.0,22.0,0.0,...,210.0,0.0,0.0,214.0,25.0,1.0,7.0,1567.0,539.0,0.0,15.0,4.0,0.0,44.0,0.0,14.0,8.0,5.0,38.0,0.0,16.0,0.0,1815.0,5.0,0.0,135.0,9997.0,30.0,47843.0,306.0,0.0,0.0,878.0,0.0,0.0,8.0,0.0,1662.0,64.0,37.0


## Make a dataframe of the 14 day notification rate per 100_000 population

In [13]:
df_14_day=pd.DataFrame()
for col in df_pvt.columns:
  df_14_day[col]=df_pvt[col].rolling(window=14).sum()

df_14_day.fillna(0., inplace=True)

for col in df_14_day.columns[1:]:
  popn=(df.loc[(df.date=='2020-10-01') & (df.iso_code==col), 'population']/100000).values
  if len(popn) > 0:
    df_14_day[col]=df_14_day[col]/popn
  else:
    print (col)

df_14_day = df_14_day.T
df_14_day.head().T

SLB


,NaN,ABW,AFG,AGO,AIA
2019-12-31,0.0,0.000000,0.000000,0.000000,0.0
2020-01-01,0.0,0.000000,0.000000,0.000000,0.0
2020-01-02,0.0,0.000000,0.000000,0.000000,0.0
2020-01-03,0.0,0.000000,0.000000,0.000000,0.0
2020-01-04,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...
2020-10-15,0.0,273.495307,1.947167,5.196818,0.0
2020-10-16,0.0,268.812169,1.903498,5.701895,0.0
2020-10-17,0.0,235.093569,2.011388,6.030499,0.0
2020-10-18,0.0,215.424386,2.168086,6.118735,0.0


In [6]:
df_14_day.loc['ESP'].tail(10) # Spain no data on weekends, delivered too late for Fridays

2020-10-10    309.339474
2020-10-11    309.339474
2020-10-12    300.936056
2020-10-13    294.973030
2020-10-14    297.013463
2020-10-15    305.352717
2020-10-16    313.610695
2020-10-17    313.610695
2020-10-18    313.610695
2020-10-19    263.391234
Name: ESP, dtype: float64

## Colour scales

In [7]:
cut_labels=['<20','20-59.9','60-119.9','120-239.9','240-479.9','>480']
cut_bins = [-1, 20., 60., 120., 240., 480., 2000.]
color_dict={'<20':'lemonchiffon','20-59.9':'yellow','60-119.9':'orange','120-239.9':'sandybrown','240-479.9':'brown','>480':'red'}

iso_3166=pd.read_csv('https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/slim-3/slim-3.csv',
                     dtype={'name':'str','alpha-3':'str','country-code':'float64'})

cells = pd.read_csv("https://raw.githubusercontent.com/mattdzugan/World-Population-Cartogram/master/data/year_2018__cell_500k/squares_and_triangles/cells.csv")

images=[]

## Plot for each day

In [8]:
for date in df_14_day.columns[[-1*(1+i*1) for i in range((pd.to_datetime(df_14_day.columns[-1])-pd.to_datetime('2020-04-30')).days)][::-1]]:
#for date in [df_14_day.columns[-1]]:
  one_day_df = pd.DataFrame(df_14_day[date])
  one_day_df['count'] = pd.cut(one_day_df[date], bins=cut_bins, labels=cut_labels)
  one_day_df['color'] = one_day_df['count'].map(color_dict)
  one_day_df = one_day_df.reset_index().rename(columns={'index':'CountryCode'})
  one_day_df = one_day_df.merge(iso_3166, left_on='CountryCode', right_on='alpha-3', how='left')
  one_day_df['country-code'] = one_day_df['country-code'].fillna(0).astype('int')

  cells_day = cells.merge(one_day_df[['country-code','color']], left_on='CountryCode', right_on='country-code', how='left')
  cells_day.color.fillna('lemonchiffon',inplace=True)

  fig = plt.figure(figsize=(16,7))
  ax = fig.add_subplot(111, aspect='equal')
  plt.xlim([0, max(cells_day["X"]+1)])
  plt.ylim([0, max(cells_day["Y"]+1)])
  n=cells_day.shape[0]
  patches = []
  for i in range(0,n):
    patches.append(Rectangle((cells_day.loc[i,"X"]+.5, cells_day.loc[i,"Y"]+.5),1,1))
  ax.add_collection(PatchCollection(patches, color=cells_day.color, alpha=0.2))

  sq1 = Line2D([0], [0], linestyle="none", marker="s", alpha=0.4, markersize=8, markerfacecolor="lemonchiffon")
  sq2 = Line2D([0], [0], linestyle="none", marker="s", alpha=0.4, markersize=8, markerfacecolor="yellow")
  sq3 = Line2D([0], [0], linestyle="none", marker="s", alpha=0.4, markersize=8, markerfacecolor="orange")
  sq4 = Line2D([0], [0], linestyle="none", marker="s", alpha=0.4, markersize=8, markerfacecolor="sandybrown")
  sq5 = Line2D([0], [0], linestyle="none", marker="s", alpha=0.4, markersize=8, markerfacecolor="brown")
  sq6 = Line2D([0], [0], linestyle="none", marker="s", alpha=0.4, markersize=8, markerfacecolor="red")

  plt.legend((sq1, sq2, sq3, sq4, sq5, sq6), ('<20 per 100 000','20-59.9','60-119.9','120-239.9','240-479.9','>480'),
             numpoints=1, loc=(.0,.77), title='1 cell: 500 000 people\n\nRate', fontsize=10, frameon=False)
  title = date+'\n14-day COVID19 case notification rate per 100 000 population'
  plt.title(title)
  plt.axis('off')
  plt.figtext(0.5, 0.01, "Sources: 'Our World in Data' and https://github.com/mattdzugan/World-Population-Cartogram", ha='center')
  plt.tight_layout()
  plt.savefig('./world_cartogram/pngs_world/'+date, dpi=150)
  #plt.show()
  plt.close()

# Write contents of the 'pngs' directory to a .gif
(very slow)

In [9]:
!pip install pygifsicle 
!apt install gifsicle

  Created wheel for pygifsicle: filename=pygifsicle-1.0.1-cp36-none-any.whl size=4062 sha256=5019f675f3a37127caa82ebe4165fdc874777a9a7cadf1429f601684c819e5a2
  Stored in directory: /root/.cache/pip/wheels/7c/27/d9/591826fc20e81ffdd7f95ac77ce0666a13fbb7b5780c7862df
Successfully built pygifsicle
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  gifsicle
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 134 kB of archives.
After this operation, 385 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 gifsicle amd64 1.91-2 [134 kB]
Fetched 134 kB in 1s (235 kB/s)
Selecting previously unselected package gifsicle.
(Reading database ... 144611 files and directories currently installed.)
Preparing to unpack .../gifsicle_1.91-2_amd64.deb ...
Unpacking gifsicle (1.91-2) ...
Setting up gifsicle (1.91-2) ...
Processing triggers fo

In [10]:
import os
import imageio
from pygifsicle import optimize

In [11]:
png_dir = '/content/drive/My Drive/TrueCue/world_cartogram/pngs_world/'
images = []

for file_name in sorted(os.listdir(png_dir)):
    if file_name.endswith('.png'):
        file_path = os.path.join(png_dir, file_name)
        images.append(imageio.imread(file_path))

imageio.mimwrite('./world_cartogram/world.gif', images, format='.gif', fps=10, loop=0)
optimize('./world_cartogram/world.gif') 